In [1]:
import os
os.chdir("/home/gayoung.lee/공모전/2021빅콘테스트")
os.getcwd()

'/home/gayoung.lee/공모전/2021빅콘테스트'

In [2]:
import pandas as pd
fw = pd.read_csv("./2021빅콘테스트_데이터분석분야_퓨처스리그_ECO제주_데이터_210811_update/01_제공데이터_update(210811)/01_음식물쓰레기_FOOD_WASTE_210811_update.CSV", encoding="cp949")
card = pd.read_csv("./2021빅콘테스트_데이터분석분야_퓨처스리그_ECO제주_데이터_210811_update/01_제공데이터_update(210811)/04_음식관련 카드소비_CARD_SPENDING.CSV", encoding="cp949")

/home/gayoung.lee/anaconda3/envs/teqz/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3166: DtypeWarning: Columns (2) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [3]:
import tqdm
import numpy as np

In [4]:
cluster = pd.read_csv("./데이터분석/클러스터링.csv")
cluster.head()

,행정동코드,행정동명,관광인구,거주인구,cluster,spot
0,50110590,건입동,Z형,1,7,제주
1,50110256,구좌읍,V형,2,5,성산
2,50130253,남원읍,Z형,1,7,서귀포
3,50110660,노형동,A형,3,3,제주
4,50130590,대륜동,Z형,3,9,서귀포


In [5]:
def year_month(x):
    x['year_month'] = x['base_date'].str.slice(start=0,stop=7)

year_month(card)
year_month(fw)

In [6]:
card1 = card.groupby(['emd_nm', 'year_month','mct_cat_nm']).sum().reset_index()
card2 = pd.merge(card1, cluster, how='outer', left_on='emd_nm', right_on='행정동명')
card2 = card2.drop(['행정동명','관광인구','거주인구','행정동코드','spot'], axis=1)
card3 = card2.groupby(['year_month','mct_cat_nm','cluster']).sum().reset_index()


In [7]:
fw1 = fw.groupby(['emd_nm', 'year_month']).sum().reset_index()
fw2 = pd.merge(fw1, cluster, how='outer', left_on='emd_nm', right_on='행정동명')
fw2 = fw2.drop(['행정동명','관광인구','거주인구','행정동코드','spot'], axis=1)
fw_cl = fw2.drop(['emd_nm','em_cnt','pay_amt'], axis=1)


In [8]:
fw3 = fw2.groupby(['year_month','cluster']).sum().reset_index()


In [11]:
import matplotlib.pyplot as plt
import scipy.cluster.hierarchy as sch
from scipy.cluster.hierarchy import fcluster

In [12]:
cluster_list = set(cluster.cluster)
card_cat_list = set(card3.mct_cat_nm)
corr_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])
card_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])


for i in range(len(cluster_list)):
    card_cl = card3[card3['cluster']==i+1]

    #클러스터 별 카드 카테고리 간 상관계수
    tmp = pd.DataFrame(columns=sorted(card_cat_list))
    for j in card_cat_list:
        tmp1 = card_cl[card_cl['mct_cat_nm']==j]
        tmp.loc[:,j] = tmp1.T.loc['use_amt'].reset_index(drop=True).astype(float)
    corr1 = tmp.corr()

    # 상관계수 바탕으로 카드 카테고리 클러스터링
    pdist = sch.distance.pdist(corr1.values)
    clust=pd.DataFrame(sch.fcluster(sch.linkage(corr1 ,method='complete'), 3, criterion='maxclust'))
    clust.columns = ['card_cluster']
    clust['mct_cat_nm'] = sorted(card_cat_list)

    # 카드 카테고리 클러스터 별로 음식물쓰레기배출량&전체인구수 상관관계
    cardcl = pd.merge(card_cl, clust, how='outer', on='mct_cat_nm')
    cardcl2 = cardcl.groupby(['year_month','card_cluster','cluster']).sum().reset_index()
    fw_cl = fw3[fw3['cluster']==i+1]
    fwcard_cl = pd.merge(cardcl2, fw_cl, how='outer',on=['cluster','year_month'])

    tmp_corr_list = []
    for k in set(fwcard_cl['card_cluster']):
        temp = fwcard_cl[fwcard_cl['card_cluster']==k]
        temp_corr = temp[['use_amt','em_g']].corr()
        tmp_corr_list.append(round(temp_corr.iloc[0,1],2))
    
 
    corr_cluster.iloc[i,:] = tmp_corr_list


    # 카드 클러스터링 결과 정리
    card_cl_dict = {}
    for l in set(clust['card_cluster']):
        card_cl_dict[l] = clust[clust['card_cluster']==l]['mct_cat_nm'].values.tolist()
    
    card_cluster.loc[i] = card_cl_dict
    
corr_cluster




,1,2,3
0,-0.49,0.59,0.37
1,0.59,-0.45,-0.08
2,0.38,0.08,-0.41
3,0.32,0.09,0.43
4,0.69,0.21,-0.46
5,0.84,0.3,-0.42
6,0.39,0.19,-0.25
7,0.85,0.55,-0.58
8,0.18,0.53,0.35


In [ ]:
card_cluster

,1,2,3
0,"[농축수산물, 배달, 식품]","[부페, 아시아음식, 양식, 패스트푸드, 한식]","[간식, 마트/슈퍼마켓, 주점및주류판매]"
1,"[마트/슈퍼마켓, 부페, 아시아음식, 양식, 패스트푸드, 한식]","[배달, 식품]","[간식, 농축수산물, 주점및주류판매]"
2,"[부페, 아시아음식, 양식, 패스트푸드, 한식]","[간식, 농축수산물, 마트/슈퍼마켓, 식품, 주점및주류판매]",[배달]
3,"[농축수산물, 배달, 식품]","[간식, 부페]","[마트/슈퍼마켓, 아시아음식, 양식, 주점및주류판매, 패스트푸드, 한식]"
4,"[아시아음식, 양식, 주점및주류판매, 한식]","[간식, 농축수산물, 마트/슈퍼마켓, 배달, 식품, 패스트푸드]",[부페]
5,"[간식, 농축수산물, 마트/슈퍼마켓, 배달, 식품, 주점및주류판매, 패스트푸드]","[아시아음식, 양식, 한식]",[부페]
6,"[부페, 아시아음식, 양식, 패스트푸드, 한식]","[간식, 농축수산물, 마트/슈퍼마켓, 식품, 주점및주류판매]",[배달]
7,"[농축수산물, 배달]","[간식, 마트/슈퍼마켓, 식품, 아시아음식, 양식, 주점및주류판매, 패스트푸드, 한식]",[부페]
8,"[부페, 아시아음식, 양식, 패스트푸드, 한식]","[간식, 농축수산물, 마트/슈퍼마켓, 식품, 주점및주류판매]",[배달]


In [18]:
#lag

cluster_list = set(cluster.cluster)
card_cat_list = set(card3.mct_cat_nm)
corr_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])
card_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])


for i in range(len(cluster_list)):
    card_cl = card3[card3['cluster']==i+1]

    #클러스터 별 카드 카테고리 간 상관계수
    tmp = pd.DataFrame(columns=sorted(card_cat_list))
    for j in card_cat_list:
        tmp1 = card_cl[card_cl['mct_cat_nm']==j]
        tmp.loc[:,j] = tmp1.T.loc['use_amt'].reset_index(drop=True).astype(float)
    corr1 = tmp.corr()

    # 상관계수 바탕으로 카드 카테고리 클러스터링
    pdist = sch.distance.pdist(corr1.values)
    clust=pd.DataFrame(sch.fcluster(sch.linkage(corr1 ,method='complete'), 3, criterion='maxclust'))
    clust.columns = ['card_cluster']
    clust['mct_cat_nm'] = sorted(card_cat_list)

    # 카드 카테고리 클러스터 별로 음식물쓰레기배출량&전체인구수 상관관계
    cardcl = pd.merge(card_cl, clust, how='outer', on='mct_cat_nm')
    cardcl2 = cardcl.groupby(['year_month','card_cluster','cluster']).sum().reset_index()
    fw_cl = fw3[fw3['cluster']==i+1]
    fwcard_cl = pd.merge(cardcl2, fw_cl, how='outer',on=['cluster','year_month'])

    tmp_corr_list = []
    for k in set(fwcard_cl['card_cluster']):
        temp = fwcard_cl[fwcard_cl['card_cluster']==k]
        temp2 = pd.DataFrame()
        temp2['em_g'] = temp['em_g']
        temp2['use_amt_lag'] = temp['use_amt'].shift(1)
        temp_corr = temp2[['use_amt_lag','em_g']].corr()
        tmp_corr_list.append(round(temp_corr.iloc[0,1],2))
    
 
    corr_cluster.iloc[i,:] = tmp_corr_list
    
corr_cluster

,1,2,3
0,-0.63,0.51,0.02
1,0.43,-0.54,-0.34
2,0.24,-0.28,-0.45
3,0.2,0.06,0.17
4,0.61,0.23,-0.39
5,0.73,0.26,-0.53
6,0.3,-0.04,-0.28
7,0.8,0.51,-0.57
8,0.14,0.28,0.33


In [14]:
import numpy as np

In [20]:
#lag + 차분

cluster_list = set(cluster.cluster)
card_cat_list = set(card3.mct_cat_nm)
corr_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])
card_cluster = pd.DataFrame(index=range(0,len(cluster_list)), columns= [1,2,3])


for i in range(len(cluster_list)):
    card_cl = card3[card3['cluster']==i+1]

    #클러스터 별 카드 카테고리 간 상관계수
    tmp = pd.DataFrame(columns=sorted(card_cat_list))
    for j in card_cat_list:
        tmp1 = card_cl[card_cl['mct_cat_nm']==j]
        tmp.loc[:,j] = tmp1.T.loc['use_amt'].reset_index(drop=True).astype(float)
    corr1 = tmp.corr()

    # 상관계수 바탕으로 카드 카테고리 클러스터링
    pdist = sch.distance.pdist(corr1.values)
    clust=pd.DataFrame(sch.fcluster(sch.linkage(corr1 ,method='complete'), 3, criterion='maxclust'))
    clust.columns = ['card_cluster']
    clust['mct_cat_nm'] = sorted(card_cat_list)

    # 카드 카테고리 클러스터 별로 음식물쓰레기배출량&전체인구수 상관관계
    cardcl = pd.merge(card_cl, clust, how='outer', on='mct_cat_nm')
    cardcl2 = cardcl.groupby(['year_month','card_cluster','cluster']).sum().reset_index()
    fw_cl = fw3[fw3['cluster']==i+1]
    fwcard_cl = pd.merge(cardcl2, fw_cl, how='outer',on=['cluster','year_month'])

    tmp_corr_list = []
    for k in set(fwcard_cl['card_cluster']):
        temp = fwcard_cl[fwcard_cl['card_cluster']==k]
        temp2 = pd.DataFrame()
        temp2['em_g_diff'] = np.diff(temp['em_g'],1)
        temp2['use_amt_diff'] = np.diff(temp['use_amt'],1)
        temp2['use_amt_diff_lag'] = temp2['use_amt_diff'].shift(1)
        temp_corr = temp2[['use_amt_diff_lag','em_g_diff']].corr()
        tmp_corr_list.append(round(temp_corr.iloc[0,1],2))
    
 
    corr_cluster.iloc[i,:] = tmp_corr_list
    
corr_cluster

,1,2,3
0,-0.27,-0.14,-0.37
1,-0.24,-0.31,-0.26
2,-0.26,-0.41,-0.22
3,-0.16,-0.25,-0.12
4,-0.07,0.1,0.28
5,-0.53,-0.28,-0.45
6,-0.06,-0.23,-0.28
7,-0.16,-0.09,0.05
8,-0.15,-0.34,-0.26
